# Keras Tuner

Keras Tuner (https://keras-team.github.io/keras-tuner/) is a python library designed to simplify the process of hyperparameter optimization in a Keras model. 

This notebook illustrates its use with a simple example.

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 4.3MB/s 


Imports:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from time import time
import shutil

Load data:

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Normalize:

In [ ]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Create a *hypermodel*, a function that generates a Keras model that depends on a set of hyperparameters we want to *play* with. The hyperparameters are sampled from the ``hp`` argument. 

In this example we just play with the regularization parameter in the hidden layer:

In [ ]:
def model_builder(hp):
  hp_lambda = hp.Choice('lambda', values = [1.0, 0.1, 0.01, 0.001, 0.0001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = 50, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Clear logs directory:

In [ ]:
!rm -rf my_dir/intro_to_kt/

Create a hyperparameter ``tuner``. There are several types:

- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn

The easiest thing is to perform a ``RandomSearch``. When creating a ``tuner`` we must indicate:

- The hypermodel.
- The variable to optimize.
- The number of trials.
- The number of runs per trial.

In [8]:
tuner = kt.RandomSearch(model_builder, 
                        objective='val_acc',
                        max_trials=10,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

This is a search space summary:

In [9]:
tuner.search_space_summary()

Launch the hyperparameter search:

In [10]:
tuner.search(img_train, label_train,
             epochs=1,
             validation_data=(img_test, label_test))

1875/1875 [==============================] - 2s 1ms/step - loss: 0.6009 - acc: 0.8913 - val_loss: 0.4343 - val_acc: 0.9162


1875/1875 [==============================] - 2s 1ms/step - loss: 0.4040 - acc: 0.9084 - val_loss: 0.2706 - val_acc: 0.9425


1875/1875 [==============================] - 2s 1ms/step - loss: 0.3356 - acc: 0.9089 - val_loss: 0.1959 - val_acc: 0.9469


1875/1875 [==============================] - 2s 1ms/step - loss: 1.0211 - acc: 0.8504 - val_loss: 0.7287 - val_acc: 0.8612


INFO:tensorflow:Oracle triggered exit


Select the best model: 

In [11]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(img_test, label_test)

313/313 [==============================] - 0s 631us/step - loss: 0.1959 - acc: 0.9469


[0.19588318467140198, 0.9469000101089478]

Results summary:

In [12]:
tuner.results_summary()

Get best model parameters and re-train with the whole training set:

In [13]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3277 - acc: 0.9121 - val_loss: 0.1979 - val_acc: 0.9466
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1786 - acc: 0.9532 - val_loss: 0.1563 - val_acc: 0.9601
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1460 - acc: 0.9635 - val_loss: 0.1414 - val_acc: 0.9640
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1276 - acc: 0.9693 - val_loss: 0.1286 - val_acc: 0.9693
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1166 - acc: 0.9729 - val_loss: 0.1268 - val_acc: 0.9686
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1089 - acc: 0.9760 - val_loss: 0.1203 - val_acc: 0.9717
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1019 - acc: 0.9780 - val_loss: 0.1218 - val_acc: 0.9709
Epoch 8/10
1875/1875 [==============================] - 2s 1ms

# Second example

Fit the learning rate, the regularization parameter and the number of neurons in the hidden layer with a *hyperband* tuner (https://arxiv.org/pdf/1603.06560.pdf):

Hypermodel:

In [14]:
def model_builder_2(hp):
  hp_lambda = hp.Choice('lambda', values = [0.001, 0.0001]) 
  hp_units = hp.Int('units', min_value = 32, max_value = 128, step = 32)
  hp_learning_rate = hp.Choice('learning_rate', values = [1.0, 0.1, 0.01, 0.001]) 
  
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu', kernel_regularizer=keras.regularizers.l2(hp_lambda)))
  model.add(keras.layers.Dense(10, activation="softmax"))

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])    
  
  return model

Clear logs folder:

In [15]:
!rm -rf my_dir/intro_hyperband/

Tuner:

In [16]:
tuner = kt.Hyperband(model_builder_2,
                     objective = 'val_acc', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_hyperband')  

Search space:

In [18]:
tuner.search_space_summary()

Search:

In [19]:
tuner.search(img_train, label_train,
             epochs=10,
             validation_data=(img_test, label_test))

Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 1.5130 - acc: 0.5760 - val_loss: 1.5470 - val_acc: 0.5517
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 1.5197 - acc: 0.5187 - val_loss: 1.4423 - val_acc: 0.5561


Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3090 - acc: 0.9178 - val_loss: 0.1821 - val_acc: 0.9503
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1628 - acc: 0.9584 - val_loss: 0.1460 - val_acc: 0.9650


Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 8.8950 - acc: 0.1015 - val_loss: 2.5530 - val_acc: 0.1028
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4061 - acc: 0.1011 - val_loss: 2.4932 - val_acc: 0.0974


Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5222 - acc: 0.9036 - val_loss: 0.4574 - val_acc: 0.9154
Epoch 2/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4537 - acc: 0.9203 - val_loss: 0.4804 - val_acc: 0.8993


Epoch 1/2
1875/1875 [==============================] - 3s 1ms/step - loss: 1.4630 - acc: 0.6097 - val_loss: 1.5053 - val_acc: 0.5253
Epoch 2/2
1875/1875 [==============================] - 3s 1ms/step - loss: 1.6268 - acc: 0.4727 - val_loss: 1.5235 - val_acc: 0.4804


...

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 29.6764 - acc: 0.1090 - val_loss: 2.3870 - val_acc: 0.0958
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4093 - acc: 0.0987 - val_loss: 2.4512 - val_acc: 0.0974
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4141 - acc: 0.1022 - val_loss: 2.4752 - val_acc: 0.1135
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4068 - acc: 0.1002 - val_loss: 2.3919 - val_acc: 0.0958
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4086 - acc: 0.1013 - val_loss: 2.3566 - val_acc: 0.0982
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4146 - acc: 0.1004 - val_loss: 2.3558 - val_acc: 0.0974
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 2.4105 - acc: 0.1021 - val_loss: 2.3804 - val_acc: 0.0982
Epoch 8/10
1875/1875 [==============================] - 3s 2m

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3652 - acc: 0.9009 - val_loss: 0.2370 - val_acc: 0.9358
Epoch 2/10
1875/1875 [==============================] - 2s 945us/step - loss: 0.2119 - acc: 0.9440 - val_loss: 0.1883 - val_acc: 0.9505
Epoch 3/10
1875/1875 [==============================] - 2s 946us/step - loss: 0.1730 - acc: 0.9551 - val_loss: 0.1627 - val_acc: 0.9588
Epoch 4/10
1875/1875 [==============================] - 2s 969us/step - loss: 0.1512 - acc: 0.9610 - val_loss: 0.1548 - val_acc: 0.9615
Epoch 5/10
1875/1875 [==============================] - 2s 975us/step - loss: 0.1371 - acc: 0.9659 - val_loss: 0.1496 - val_acc: 0.9616
Epoch 6/10
1875/1875 [==============================] - 2s 967us/step - loss: 0.1274 - acc: 0.9686 - val_loss: 0.1457 - val_acc: 0.9644
Epoch 7/10
1875/1875 [==============================] - 2s 965us/step - loss: 0.1205 - acc: 0.9703 - val_loss: 0.1332 - val_acc: 0.9682
Epoch 8/10
1875/1875 [============================

Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3785 - acc: 0.9231 - val_loss: 0.2488 - val_acc: 0.9541
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2338 - acc: 0.9574 - val_loss: 0.2145 - val_acc: 0.9617
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2008 - acc: 0.9640 - val_loss: 0.1816 - val_acc: 0.9685
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1870 - acc: 0.9679 - val_loss: 0.1997 - val_acc: 0.9619
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1766 - acc: 0.9694 - val_loss: 0.1729 - val_acc: 0.9700
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1699 - acc: 0.9715 - val_loss: 0.1773 - val_acc: 0.9680
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1637 - acc: 0.9724 - val_loss: 0.1617 - val_acc: 0.9726
Epoch 8/10
1875/1875 [==============================] - 3s 2ms

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 8.3328 - acc: 0.1067 - val_loss: 2.3426 - val_acc: 0.0958
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4041 - acc: 0.1010 - val_loss: 2.4561 - val_acc: 0.0892
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4090 - acc: 0.1001 - val_loss: 2.3905 - val_acc: 0.0958
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4107 - acc: 0.1022 - val_loss: 2.3886 - val_acc: 0.0980
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4138 - acc: 0.0999 - val_loss: 2.4209 - val_acc: 0.1028
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4102 - acc: 0.0995 - val_loss: 2.4056 - val_acc: 0.0980
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 2.4078 - acc: 0.1035 - val_loss: 2.4054 - val_acc: 0.1135
Epoch 8/10
1875/1875 [==============================] - 2s 1ms

INFO:tensorflow:Oracle triggered exit


Best model:

In [20]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(img_test, label_test)

313/313 [==============================] - 0s 768us/step - loss: 0.1091 - acc: 0.9760


[0.10910527408123016, 0.9760000109672546]

Best hyperparameters:

In [21]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lambda': 0.0001,
 'learning_rate': 0.001,
 'tuner/bracket': 2,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/round': 2,
 'tuner/trial_id': '7dd6418bff7c01eee17184418d381fc7',
 'units': 96}

Retrain best model:

In [22]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2840 - acc: 0.9248 - val_loss: 0.1766 - val_acc: 0.9528
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1493 - acc: 0.9640 - val_loss: 0.1343 - val_acc: 0.9666
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1201 - acc: 0.9727 - val_loss: 0.1292 - val_acc: 0.9682
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1064 - acc: 0.9771 - val_loss: 0.1161 - val_acc: 0.9731
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0960 - acc: 0.9808 - val_loss: 0.1114 - val_acc: 0.9751
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0891 - acc: 0.9823 - val_loss: 0.1145 - val_acc: 0.9736
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0839 - acc: 0.9844 - val_loss: 0.1066 - val_acc: 0.9763
Epoch 8/10
1875/1875 [==============================] - 3s 1ms